# Leaky Building Pit

Testing the LeakyBuildingPit.

1. [Single layer model](#1-single-layer-model)
2. [Single layer model with well](#2-single-layer-model-with-well)
3. [Two-layer model](#3-two-layer-model)
4. [Two layer model with well](#4-two-layer-model-with-well)

In [ ]:
import timml as tml
import numpy as np
import matplotlib.pyplot as plt

## 1. Single layer model

In [ ]:
# model parameters
kh = 10  # m/day
kv = 0.25 * kh  # m/day

ctop = 1000.  # resistance top leaky layer in days

ztop = 0. # surface elevation
zbot = -20.  # bottom elevation of the model

h_bem = -1.0  # m, top boundary in pit
res = 100.0  # days, resistance of sheetpile wall

dx = 40.  # length building pit in m
dy = 30.   # width building pit in m

xy = [
    (-dx / 2, -dy / 2),
    (dx / 2, -dy / 2),
    (dx / 2, dy / 2),
    (-dx / 2, dy / 2),
    (-dx / 2, -dy / 2),
]

# get data ready for timml
z = np.array([ztop+1, ztop, zbot])
dz = z[1::2] - z[2::2]
kh_arr = kh * np.ones(dz.shape)
c = np.r_[np.array([ctop]), dz[:-1]/(2*kv) + dz[1:]/(2*kv)]

In [ ]:
# reference solution for res -> inf
ml0_ref = tml.ModelMaq(kaq=kh_arr, z=z, c=c, topboundary="semi", hstar=0.0)
bpit = tml.BuildingPit(
    ml0_ref,
    xy,
    kaq=kh_arr,
    z=z,
    topboundary="semi",
    hstar=h_bem,
    c=c,
    layers=[0],
)
ml0_ref.solve()


In [ ]:
ml0 = tml.ModelMaq(kaq=kh_arr, z=z, c=c, topboundary="semi", hstar=0.0)
bpit = tml.LeakyBuildingPit(
    ml0,
    xy,
    kaq=kh_arr,
    z=z,
    topboundary="semi",
    hstar=h_bem,
    c=c,
    layers=[0],
    res=res,
)
ml0.solve()


In [ ]:
ml0.contour(win=(-dx/2, dx/2, -dy/2, dy/2), ngr=51, levels=30, decimals=3, layers=[0]);

In [ ]:
x = np.linspace(-100, 100, 100)
h0 = ml0.headalongline(x, np.zeros_like(x))
h0_ref = ml0_ref.headalongline(x, np.zeros_like(x))

In [ ]:
plt.plot(x, h0[0], label=f"res={res:.0f} d")
# plt.plot(x, h0_ref[0], label="res=$\infty$ d")
plt.axvline(-dx/2, ls="dashed", color="k", lw=1.0)
plt.axvline(dx/2, ls="dashed", color="k", lw=1.0)
plt.grid(True)
plt.legend(loc=(0, 1), frameon=False, ncol=2);

In [ ]:
Qn0 = ml0.disvecalongline(x, np.zeros_like(x))[0]

In [ ]:
plt.plot(x, Qn0[0])
plt.axvline(-dx/2, ls="dashed", color="k", lw=1.0)
plt.axvline(dx/2, ls="dashed", color="k", lw=1.0)
plt.grid(True)

## 2. Single layer model with well

In [ ]:
ml1 = tml.ModelMaq(kaq=kh_arr, z=z, c=c, topboundary="semi", hstar=0.0)

inhom = tml.LeakyBuildingPit(
    ml1,
    xy,
    kaq=kh_arr,
    z=z[1:],
    topboundary="conf",
    c=[],
    layers=[0],
    res=res,
)
well = tml.Well(ml1, 0.0, 0.0, Qw=100.0)
ml1.solve()

In [ ]:
ml1.contour(win=(-dx/2, dx/2, -dy/2, dy/2), ngr=51, levels=30, decimals=2, layers=[0]);

In [ ]:
x = np.linspace(-100, 100, 100)
h1 = ml1.headalongline(x, np.zeros_like(x))

In [ ]:
plt.plot(x, h1[0], label=f"res={res:.0f} d")
plt.axvline(-dx/2, ls="dashed", color="k", lw=1.0)
plt.axvline(dx/2, ls="dashed", color="k", lw=1.0)
plt.grid(True)
plt.legend(loc=(0, 1), frameon=False, ncol=2)
plt.ylim(top=0.05);

In [ ]:
Qn1 = ml1.disvecalongline(x, np.zeros_like(x))[0]

In [ ]:
plt.plot(x, Qn1[0])
plt.axvline(-dx/2, ls="dashed", color="k", lw=1.0)
plt.axvline(dx/2, ls="dashed", color="k", lw=1.0)
plt.grid(True)

## 3. Two-layer model

In [ ]:
# model parameters
kh = 10  # m/day
kv = 0.25 * kh  # m/day

ctop = 1000.  # resistance top leaky layer in days

ztop = 0. # surface elevation
zwall = -10.0  # bottom elevation of sheetpile wall
zbot = -20.  # bottom elevation of the model

h_bem = -1.0  # m, top boundary in pit
res = 100.0  # days, resistance of sheetpile wall

dx = 40.  # length building pit in m
dy = 30.   # width building pit in m

xy = [
    (-dx / 2, -dy / 2),
    (dx / 2, -dy / 2),
    (dx / 2, dy / 2),
    (-dx / 2, dy / 2),
    (-dx / 2, -dy / 2),
]

# get data ready for timml
z = np.array([ztop+1, ztop, zwall, zwall, zbot])
dz = z[1::2] - z[2::2]
kh_arr = kh * np.ones(dz.shape)
c = np.r_[np.array([ctop]), dz[:-1]/(2*kv) + dz[1:]/(2*kv)]

In [ ]:
# reference solution for res -> inf
ml2_ref = tml.ModelMaq(kaq=kh_arr, z=z, c=c, topboundary="semi", hstar=0.0)
bpit = tml.BuildingPit(
    ml2_ref,
    xy,
    kaq=kh_arr,
    z=z,
    topboundary="semi",
    hstar=h_bem,
    c=c,
    layers=[0],
)
ml2_ref.solve()

In [ ]:
ml2 = tml.ModelMaq(kaq=kh_arr, z=z, c=c, topboundary="semi", hstar=0.0)
bpit = tml.LeakyBuildingPit(
    ml2,
    xy,
    kaq=kh_arr,
    z=z,
    topboundary="semi",
    hstar=h_bem,
    c=c,
    layers=[0],
    res=res,
)
ml2.solve()

In [ ]:
ml2.contour(win=(-dx/2, dx/2, -dy/2, dy/2), ngr=51, levels=30, decimals=3, layers=[0, 1]);

In [ ]:
x = np.linspace(-100, 100, 100)
h2 = ml2.headalongline(x, np.zeros_like(x))
h2_ref = ml2_ref.headalongline(x, np.zeros_like(x))

In [ ]:
plt.plot(x, h2[0], label=f"res={res:.0f} d, layer 0")
plt.plot(x, h2[1], label=f"res={res:.0f} d, layer 1")
plt.plot(x, h2_ref[0], c="C0", ls="dashed", label="res=$\infty$ d, layer 0")
plt.plot(x, h2_ref[1], c="C1", ls="dashed", label="res=$\infty$ d, layer 1")
plt.axvline(-dx/2, ls="dashed", color="k", lw=1.0)
plt.axvline(dx/2, ls="dashed", color="k", lw=1.0)
plt.grid(True)
plt.legend(loc=(0, 1), frameon=False, ncol=2);

In [ ]:
Qn2 = ml2.disvecalongline(x, np.zeros_like(x))[0]
Qn2_ref = ml2_ref.disvecalongline(x, np.zeros_like(x))[0]

In [ ]:
plt.plot(x, Qn2[0], label=f"res={res:.0f} d, layer 0")
plt.plot(x, Qn2[1], label=f"res={res:.0f} d, layer 1")
plt.plot(x, Qn2_ref[0], c="C0", ls="dashed", label="res=$\infty$ d, layer 0")
plt.plot(x, Qn2_ref[1], c="C1", ls="dashed", label="res=$\infty$ d, layer 1")
plt.axvline(-dx/2, ls="dashed", color="k", lw=1.0)
plt.axvline(dx/2, ls="dashed", color="k", lw=1.0)
plt.grid(True)
plt.legend(loc=(0, 1), frameon=False, ncol=2);

## 4. Two-layer model with well

In [ ]:
# reference solution for res -> inf
ml3_ref = tml.ModelMaq(kaq=kh_arr, z=z, c=c, topboundary="semi", hstar=0.0)
bpit = tml.BuildingPit(
    ml3_ref,
    xy,
    kaq=kh_arr,
    z=z[1:],
    topboundary="conf",
    c=c[1:],
    layers=[0],
)
well = tml.Well(ml3_ref, 0.0, 0.0, Qw=100.0, layers=[0])
ml3_ref.solve()

In [ ]:
ml3 = tml.ModelMaq(kaq=kh_arr, z=z, c=c, topboundary="semi", hstar=0.0)
bpit = tml.LeakyBuildingPit(
    ml3,
    xy,
    kaq=kh_arr,
    z=z[1:],
    topboundary="conf",
    c=c[1:],
    layers=[0],
    res=res,
    # order=5,
    # ndeg=5,
)
well = tml.Well(ml3, 0.0, 0.0, Qw=100.0, layers=[0])
ml3.solve()

In [ ]:
ml3.contour(win=(-dx/2, dx/2, -dy/2, dy/2), ngr=51, levels=30, decimals=3, layers=[0, 1]);

In [ ]:
x = np.linspace(-100, 100, 100)
h3 = ml3.headalongline(x, np.zeros_like(x))
h3_ref = ml3_ref.headalongline(x, np.zeros_like(x))

In [ ]:
plt.plot(x, h3[0], label=f"res={res:.0f} d, layer 0")
plt.plot(x, h3[1], label=f"res={res:.0f} d, layer 1")
plt.plot(x, h3_ref[0], c="C0", ls="dashed", label="res=$\infty$ d, layer 0" )
plt.plot(x, h3_ref[1], c="C1", ls="dashed", label="res=$\infty$ d, layer 1" )
plt.axvline(-dx/2, ls="dashed", color="k", lw=1.0)
plt.axvline(dx/2, ls="dashed", color="k", lw=1.0)
plt.grid(True)
plt.legend(loc=(0, 1), frameon=False, ncol=2);

In [ ]:
Qn3 = ml3.disvecalongline(x, np.zeros_like(x))[0]
Qn3_ref = ml3_ref.disvecalongline(x, np.zeros_like(x))[0]

In [ ]:
plt.plot(x, Qn3[0], label=f"res={res:.0f} d, layer 0")
plt.plot(x, Qn3[1], label=f"res={res:.0f} d, layer 1")
plt.plot(x, Qn3_ref[0], c="C0", ls="dashed", label="res=$\infty$ d, layer 0")
plt.plot(x, Qn3_ref[1], c="C1", ls="dashed", label="res=$\infty$ d, layer 1")
plt.axvline(-dx/2, ls="dashed", color="k", lw=1.0)
plt.axvline(dx/2, ls="dashed", color="k", lw=1.0)
plt.grid(True)
plt.legend(loc=(0, 1), frameon=False, ncol=2);